In [1]:
%aiida

Loaded AiiDA DB environment.

In [2]:
import nglview
import numpy as np
import ipywidgets as ipw
from collections import OrderedDict
from IPython.display import display, clear_output, HTML
import nglview

from aiidalab_widgets_base import CodeDropdown, SubmitButtonWidget, MetadataWidget, StructureBrowserWidget
from aiida.engine import submit, run, run_get_node


#from apps.surfaces.widgets.computer_code_selection import ComputerCodeDropdown
from apps.surfaces.widgets.dft_details_dev import DFTDetails
from apps.surfaces.widgets.viewer_details import ViewerDetails
from apps.surfaces.widgets.slab_validity import slab_is_valid
from apps.surfaces.widgets.suggested_param import suggested_parameters
#from apps.surfaces.widgets.submit_button import SubmitButton

from ase.data import vdw_radii

from apps.surfaces.widgets import analyze_structure


ReplicaWorkChain = WorkflowFactory('replica') 

from apps.surfaces.widgets.collective_variable_widget import CollectiveVariableWidget

_ColormakerRegistry()

In [3]:
%%javascript
IPython.OutputArea.prototype._should_scroll = function(lines) {
    return false;
}

<IPython.core.display.Javascript object>

In [4]:
## GENERAL PARAMETERS

the_workchain = ReplicaWorkChain
job_details = {'workchain':'ReplicaWorkChain'}

atoms = None

In [5]:
## WIDGETS MONITOR FUNCTIONS
def on_struct_change(c):
    global atoms
    if not struct_browser.results.value:
        return
    job_details['structure']=struct_browser.results.value
    node = struct_browser.results.value
    
    atoms = node.get_ase()
    atoms.pbc = [1, 1, 1]
    
    slab_analyzed = analyze_structure.analyze(atoms)
    job_details['slab_analyzed'] = slab_analyzed
    dft_details_widget.reset()
    
    guess_calc_params(slab_analyzed)
    
    viewer_widget.setup(atoms, slab_analyzed)
    
    with mol_ids_info_out:
        clear_output()
        print(slab_analyzed['summary'])

def on_fixed_atoms_btn_click(c):
    if dft_details_widget.btn_fixed_pressed:
        viewer_widget.show_fixed(dft_details_widget.fixed_atoms.value)
    else:
        viewer_widget.show_fixed("")
        
def guess_calc_params(slab_analyzed):
    method = dft_details_widget.calc_type.value
    valid_slab, msg = slab_is_valid(slab_analyzed,method)
    if valid_slab:        
        atoms_to_fix,num_machines=suggested_parameters(slab_analyzed,method)
        dft_details_widget.reset(fixed_atoms=atoms_to_fix,calc_type=method)
        num_nodes_selector.value = num_machines
    else:
        print(msg)

## Step 1: Select initial structure

In [6]:
## DISPLAY WIDGETS AND DEFINE JOB PARAMETERS

##STRUCTURE
struct_browser = StructureBrowserWidget()
struct_browser.results.observe(on_struct_change, names='value')

##VIEWER
viewer_widget = ViewerDetails()

mol_ids_info_out = ipw.Output()

display(ipw.VBox([struct_browser, viewer_widget, mol_ids_info_out]))

## Step 2: General parameters of the calculation

In [7]:
##CODE
computer_code_dropdown = CodeDropdown(input_plugin='cp2k', path_to_root="../../")

# Misc details
style = {'description_width': '120px'}
layout = {'width': '70%'}
num_nodes_selector = ipw.IntText(
    value=1,
    description='# Nodes',
    style=style, layout=layout)
calc_name_selector = ipw.Text(
    description='Calculation Name: ',
    placeholder='A great name.',
    style=style, layout=layout)

##DFT
dft_details_widget = DFTDetails(job_details=job_details)    
dft_details_widget.btn_fixed_atoms.on_click(on_fixed_atoms_btn_click)
dft_details_widget.calc_type.observe(lambda c: guess_calc_params(), names='value')

# display code dropdown
display(ipw.VBox([computer_code_dropdown, num_nodes_selector, calc_name_selector, dft_details_widget]))

## Step 3: Define collective variable
The unit of the spring constant is [eV/unit of colvar^2].

Three kinds of collective variable are implemented:

**DISTANCE between two atoms**

**ANGLE_PLANE_PLANE between two planes**
- Rotation of plane defined by (P1 - P2) and (P3 - P2) WRT plane defined by a normal .

**BOND_ROTATION**
- Clockwise rotation of (P2-P1) WRT (P4-P3).

In [8]:
cv_widget = CollectiveVariableWidget()

def on_vis_cv_click(c):
    if cv_widget.validation_check() and atoms is not None:
        vis_list = cv_widget.current_cv_instance.visualization_list(atoms)
        viewer_widget.visualize_extra(vis_list)
        
        with mol_ids_info_out:
            cv_widget.current_cv_instance.print_cv(atoms)
        
cv_widget.visualize_colvar_btn.on_click(on_vis_cv_click)

display(cv_widget)

CollectiveVariableWidget(children=(Select(description='Colvar Type', layout=Layout(width='70%'), options={'DIS…

In [9]:
# BUG: MAX_FORCE IS NOT USED (hardcoded value in workchain)
# NOR IS VDW SWITCH USED

def on_submit():
    with submit_out:
        clear_output()
        if not struct_browser.results.value:
            print("Please select a structure.")
            return
        if not computer_code_dropdown.selected_code:
            print("Please select a code.")
            return

        builder = ReplicaWorkChain.get_builder()
        builder.cp2k_code = computer_code_dropdown.selected_code
        builder.structure = struct_browser.results.value
        builder.num_machines = Int(num_nodes_selector.value)
        builder.calc_name = Str(calc_name_selector.value)
        #builder.cell
        builder.fixed_atoms = Str(dft_details_widget.fixed_atoms)
        builder.colvar_targets = Str(cv_widget.colvar_targets)
        builder.target_unit = Str(cv_widget.target_unit)
        builder.spring = Float(cv_widget.spring_constant)
        builder.spring_unit = Str(cv_widget.spring_unit)
        builder.subsys_colvar = Dict(cv_widget.subsys_colvar)
        builder.calc_type = Str(dft_details_widget.calc_type.value)
        builder.mgrid_cutoff = Int(dft_details_widget.mgrid_cutoff.value)
        builder.metadata = {
            "description": calc_name_selector.value,
            "label": "ReplicaWorkChain",
        }
        print("----")
        print(builder)
        print("----")
        return builder

In [10]:
def on_submit():
    with submit_out:

        builder = ReplicaWorkChain.get_builder()
        builder.cp2k_code = load_node(975)
        builder.structure = load_node(974)
        builder.num_machines = Int(1)
        builder.calc_name = Str("c2h2 test")
        #builder.cell
        builder.fixed_atoms = Str("")
        builder.colvar_targets = Str("1.3 1.5 1.7 2.0")
        builder.target_unit = Str('angstrom')
        builder.spring = Float(30.0)
        builder.spring_unit = Str('eV/angstrom^2')
        builder.subsys_colvar = Dict(dict={'DISTANCE': {'ATOMS': '2 3 '}})
        builder.calc_type = Str('Full DFT')
        builder.mgrid_cutoff = Int(600)
        builder.metadata = {
            "description": "c2h2 test",
            "label": "ReplicaWorkChain",
        }
        print("----")
        print(builder)
        print("----")
        return builder

In [11]:
btn_submit = SubmitButtonWidget(workchain=ReplicaWorkChain,widgets_values=on_submit)
submit_out = ipw.Output()
display(btn_submit, submit_out)

SubmitButtonWidget(children=(Button(description='Submit', style=ButtonStyle()), Output()))

Output()

In [12]:
builder = ReplicaWorkChain.get_builder()
#builder.cp2k_code = load_node(975)
builder.cp2k_code = load_node(1)
builder.structure = load_node(4033)
builder.num_machines = Int(1)
builder.calc_name = Str("h2 test")
#builder.cell
builder.fixed_atoms = Str("")
builder.colvar_targets = Str("1.1 1.2 1.3")
builder.target_unit = Str('angstrom')
builder.spring = Float(30.0)
builder.spring_unit = Str('eV/angstrom^2')
builder.subsys_colvar = Dict(dict={'DISTANCE': {'ATOMS': '1 2'}})
builder.calc_type = Str('Full DFT')
builder.mgrid_cutoff = Int(600)
builder.metadata = {
    "description": "h2 test",
    "label": "ReplicaWorkChain",
}
print("----")
print(builder)
print("----")

----
{'metadata': {'description': 'h2 test', 'label': 'ReplicaWorkChain'}, 'cp2k_code': <Code: Remote code 'cp2k' on localhost, pk: 1, uuid: 8963b2e6-3418-4180-91eb-8d740dfe5e7e>, 'structure': <StructureData: uuid: 83aeecda-744a-44d0-8848-e9cf122f9632 (pk: 4033)>, 'num_machines': <Int: uuid: 2d722399-7f95-498b-9102-5697f71515e5 (unstored) value: 1>, 'calc_name': <Str: uuid: b397f56b-b0bf-4414-b2fc-82ce48c6ca5d (unstored) value: h2 test>, 'fixed_atoms': <Str: uuid: 083cf366-186c-4309-ad93-ec98a6895062 (unstored) value: >, 'colvar_targets': <Str: uuid: 3299f2c6-76f0-4531-acc6-1ef5aa3e7d30 (unstored) value: 1.1 1.2 1.3>, 'target_unit': <Str: uuid: ec2be989-b050-4cf8-9672-f0c419402879 (unstored) value: angstrom>, 'spring': <Float: uuid: 4e07c056-827b-4042-adc0-9b54f0efeee9 (unstored) value: 30.0>, 'spring_unit': <Str: uuid: 53a1aee8-869c-47ab-80cc-aa7eb9e36a41 (unstored) value: eV/angstrom^2>, 'subsys_colvar': <Dict: uuid: 98d82560-6fc4-4c26-8587-622dba90f40b (unstored)>, 'calc_type': <Str

In [13]:
node = run(ReplicaWorkChain, **builder)

11/22/2019 12:10:43 PM <17769> aiida.orm.nodes.process.workflow.workchain.WorkChainNode: [REPORT] [4345|ReplicaWorkChain|initialize]: Init generate replicas
11/22/2019 12:10:43 PM <17769> aiida.orm.nodes.process.workflow.workchain.WorkChainNode: [REPORT] [4345|ReplicaWorkChain|initialize]: #3 replicas
11/22/2019 12:10:43 PM <17769> aiida.orm.nodes.process.workflow.workchain.WorkChainNode: [REPORT] [4345|ReplicaWorkChain|next_replica]: Go to replica - 3
11/22/2019 12:10:43 PM <17769> aiida.orm.nodes.process.workflow.workchain.WorkChainNode: [REPORT] [4345|ReplicaWorkChain|next_replica]: Remaining list: ['1.1', '1.2', '1.3'] (3)
11/22/2019 12:10:43 PM <17769> aiida.orm.nodes.process.workflow.workchain.WorkChainNode: [REPORT] [4345|ReplicaWorkChain|generate_replica]: Running CP2K geometry optimization - Target: 1.1
11/22/2019 12:10:43 PM <17769> aiida.orm.nodes.process.workflow.workchain.WorkChainNode: [REPORT] [4345|ReplicaWorkChain|generate_replica]:  
11/22/2019 12:10:43 PM <17769> aii